Run the config file to authenticate the machine and query variables from Key Vault

In [52]:
%run config

Code from file 'file:///c%3A/Users/nicholas.radich/Documents/Strava_Lakehouse/config.py':
 client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 
client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 
new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")
activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 
historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 
segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 
segment_details_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmentdetailspath") 



import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'cli

Depedencies 

In [53]:
from pyspark.sql.functions import * 
from pyspark.sql.types import LongType
import pandas as pd
import requests 
import utils

API call to grab all of the acitivites within a personal account

Make API call to get all strava activites

In [54]:
my_dataset = utils.activity_api_call(access_token)

Extracts activity ids and supporting information from initial dataset

Extract activity IDs and return refined columns 

In [55]:

activity_id_DF, activity_df = utils.extract_activities(my_dataset)

Function to write datasets to storage, delta format

Write Activity_ID_DF to storage

In [56]:
#downside dataframes for testing purpose
activity_df= activity_df.limit(10)
activity_id_DF = activity_id_DF.limit(10)

In [57]:
#write the activity IDs to storage
utils.write_dataframe_to_storage(activity_id_DF,activity_id_path, "overwriteSchema","overwrite" )

Read the activities from storage

In [58]:
#read the activities from storage
stored_activity_ids = spark.read.format("delta").load(activity_id_path)

Compare what is currently in storage vs the most recent API call
If nothing is written to storage ie first run, still need to execute and write original DF to storage

Write full dataset to storage, will check to see if data already exists at path 

In [59]:
historical_activites = utils.get_historical_dataset(historical_activity_id_path, activity_df, historical_activity_id_path)

Extract those activites not currently in storage

In [60]:
#Extract activity ids from dataframes to list, to make comparison 
#activity_id_list = activity_id_DF.select('activity_ids').distinct().collect()
#from full dataset, all unique activity ids
activity_id_list = stored_activity_ids.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()

#currently what is written in storage, activity IDs
historical_activity_id_list = historical_activites.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

#make comparison between two, and find those IDs not written to storage
activity_ids_not_written_to_storage = [x for x in activity_id_list if x not in historical_activity_id_list ]

#filter original DF with those ids not currently in storage
new_activities = activity_df.filter(activity_df.activity_ids.isin(activity_ids_not_written_to_storage))

In [62]:
#append new activity IDs to the storage location
utils.write_dataframe_to_storage(new_activities, historical_activity_id_path,"mergeSchema", "append")

In [ ]:
df1 = spark.read.format("delta").load(historical_activity_id_path)

In [ ]:
#only adds in distinct activities
from pyspark.sql import functions as F
df1.orderBy(F.desc('activity_ids')).show()

In [ ]:
activity_id_DF.select('activity_id').show()